# Нормализация сцен Ландсат

Даны разновременные сцены Ландсат, покрывающие одну и ту же территорию. Яркости пикселей разных сцен изначально несравнимы между собой вследствие различий в условиях съемки. Требуется промасштабировать яркости пикселей этих сцен таким образом, чтобы появилась возможность сравнивать их между собой и отслеживать динамику изменений яркостей. Эта процедура называется относительной нормализацией.

Нормализация, применяемая в данном документе, следует методике из статьи Du Y., Teillet P. M., Cihlar J. Radiometric normalization of multitemporal high-resolution satellite images with quality control for land cover change detection //Remote sensing of Environment. – 2002. – Т. 82. – №. 1. – С. 123-134.

Основные положения статьи:

1. Если выполняются определенные условия, в частности, если нелинейные эффекты от снимка к снимку незначительны по сравнению с линейными, и искажения равномерно распределены в пространстве, то можно производить линейную нормализацию. (Более точная формулировка условия дана в следующем разделе.)
2. Для линейной нормализации следует выбрать псевдоинвариантные объекты (PIF) и линейно преобразовать яркости пикселей таким образом, чтобы статистики яркостей (среднее и дисперсия), вычисленные по выбранным PIF совпадали у различных сцен.
3. PIF можно выбирать вручную, но это долго и не объективно. В статье предлагается процедура автоматического выбора PIF. Поскольку эта часть статьи наиболее интересна для нас и на ней основана вся методика, то она рассматривается ниже в отдельном разделе.

## Методика выбора PIF: теория

### Ограничения метода

Предположим, что выполняются следующие условия:

1. Отражающая способность объектов, выбранных в качестве PIF и используемых в процедуре нормализации, является постоянной в течении анализируемого периода времени.
2. Все изменения яркостей пикселей (DN), которые фиксируются на снимках для выбранных PIF, не являются реальным изменением объектов на местности.
3. Все изменения DN в течении анализируемого периода времени линейны и пространственно однородны для всех PIF, учавствующих в процедуре нормализации.
4. Все эффекты, приводящие к изменениям DN, независимы друг от друга и их результирующая величина представляет собой линейную комбинацию отдельных эффектов.
5. Реальные изменения объектов сцены не являются массовыми.

В этом случае процедура относительной нормализации сцен может быть реализована на базе линейного преобразования. 

**Замечание.** В рассматриваемой статье предполагается, что это DN (при отосительной нормализации не обязательно переходить от DN к ToAR, поскольку этот переход линеен и, следовательно, будет учтен при нормализации), но мы будем работать не с DN, а с ToAR и данными, прошедшими топографическую коррекцию. Этот выбоор обусловлен тем, что нам необходимо произвести маскирование облачности (оно оперирует ToAR), а также тем, что в случае горных условий Дальнего Востока очень желательно произвести топографическую коррекцию, поскольку нужно нивелировать влияние склонов и положения Солнца (без этого возникнет явное противоречие с условием пространственной однородности).

### Нормализация

Пусть Q(i) -- яркость пикселя PIF номер i. Линейная нормализация подразумевает преобразование изображений 1, 2, ..., j, ..., m по следующей формуле:
$$
N_j(i) = Q_j(i)\times \alpha_i + \beta_i = Q_j(i)\times \frac{\sigma_{ref}^{(Q)}}{\sigma_j^{(Q)}} +  \overline{Q}_{ref} \frac{\sigma_{ref}^{(Q)}}{\sigma_j^{(Q)}}\times \overline{Q}_j
$$
где $N_j(i)$ - нормированное значение i-го PIF в j-м изображении, $\overline{Q}_{ref}$, $\sigma_{ref}^{(Q)}$ - желаемые статистики средних яркостей и дисперсии, рассчитанных по всем PIF (они могут быть произвольными константами или же вычислены на базе произвольного изображения), а $\overline{Q}_j$ и $\sigma_j^{(Q)}$ - средняя яркость и дисперсия для PIF в изображении j.

### Стратегии выбора PIF

На практике PIF выбираются или руками (что нам не интересно), или автоматически. Будем рассматривать второй способ. 

Поскольку выше были сформированы условия линейности, то при их выполнении мы можем расчитывать на то, что PIFы между собой связаны линейным законом. Это означает, что если мы возьмем два изображения и преобразуем их при помощи метода главных компонент, то основная информация будет сконцентрирована вдоль одной оси (первой главной компоненты), вдоль второй оси будут собраны шумы и изменения. Это обосновывает (в статье рассказывается несколько подробнее) следующую процедуру поиска PIF:

1. Очистка данных от заведомо нестабильных пикселей: (а) применить порог для удаления облачных пикселей; (б) удалить пиксели, соответствующие водным объектам (они быстро меняются); (в) удалить все пиксели, для которых разность между первым и вторым изображением выше некоторого порога (они сигнализируют о потенциальном изменении, опасно выбирать их в качестве PIF).
2. Произвести преобразование оставшихся пикселей выбранной пары изображений методом главных компонент.
3. Все пиксели, лежащие вдоль главной оси на расстоянии не большем $l$ единиц (т.е. $-l < minor < l$, где minor - вторая главная компонента) маркируются как потенциальные PIF.
4. Если выбранные PIF удовлетворяют критериям качества (см. ниже), то процедура поиска останавливается. Если нет, то уменьшается величина $l$ (а также, возможно, ужесточаются пороговые значения) и производится повторный поиск PIF при новых параметрах.
5. Вычисляются среднее и дисперсия яркостей выбранных PIF (величины $\overline{Q}_j$ и $\sigma_j^{(Q)}$), которые в дальнейшем используются при нормализации.

**Замечание.** В статье рассматриваются, по всей видимости, только одноканальные изображения или же многоканальные, но при этом каждый канал анализируется отдельно от остальных. В случае многоканальных изображений, вероятно, такой подход приведет к некторому искажению в отношении каналов: $B_k/B_m \approx N_k/N_m (?)$ (Другими словами, после нормировки спектральные вектора подвергнутся некоторому повороту; небольшой поворот не страшен, он может объясняться шумами, но нет гарантии, что этот поворот не окажется больше, чем следует). Т.е. по-хорошему, нужно доработать процедуру, предоженную в статье, для того, чтобы можно было использовать многоканальные изображения без искажений спектральных векторов объектов. (Ну или убедиться, что данная процедура не приводит к такому искажению). На текущем этапе оставим все как есть.

### Оценка качества PIF

Для оценки качества PIF авторы предлагают пройти несколько стадий/способов оценки:

1. Посчитать коэффициент корреляции между выбранными PIF. Авторы предлагают считать набор не подходящим, если коэффициент не превышает значение 0.9. Этот метод должен неплохо срабатывать на этапе выбора порогов разности изображений и/или порогов облачности и водных объектов.
2. После применения метода главных компонент оценкой качества выбранных PIF может служить величина наклона главной оси: чем ближе она к 45 градусам, тем лучше. Итеративным подбором величины $l$ можно добиться постепенного приближения наклона главной оси к желаемому.
3. Этот способ применим, если производится нормализация не одной пары, а нескольких пар снимков. Например, имеются три снимка A, B и C. Тогда можно применить метод главных компонент к парам AB, AC, BC. При этом не исключено, что параметры нормализации, полученные для разных пар могут быть разными (зная парамаметры нормализации для пар AB и BC можно получить параметры для AC, которые могут противоречить вычисленным напрямую для AC значениям). Поэтому еще одним критерием качества может служить близость параметров нормализации, вычисленных разными способами.

## Нормализация: практика

### Процедура выбора PIF

Создадим функцию, которая будет брать на вход названия двух растров и искать на них PIF, результирующие PIF будут сохранены в виде растра с заданным именем. Функция вернет словарь с описанием процесса работы:
 
 * b: коффициент регрессии между PIF (тангенс угла наклона линии регрессии), чем ближе к единице, тем лучше;
 * N: число обнаруженных PIF;
 * iterations: число выполненных итераций;
 * good_quality: True, если процедура сошлась, и False в противном случае.

**TODO:** Сейчас реализация просто даляет маску (растр MASK), если она была. По-хорошему, нужно сделать проверку того, что растр маски присутствует и восстанавливать его после работы функции.

Функция управляется несколькими параметрами. Наиболее важные из них:

* regression_eps описывает допустимое отклонение регрессионной кривой от 45 градусов наклона; если по завершении работы алгоритма тангенс угла наклона отличается от тагнеса 45 градусов более, чем на эту величину (по модулю), считаем, что алгритм не сошелся и PIF не достоверны.
* min_pixels=1000 число пикселей (PIF), при которых результат еще считается достоверным; с каждой итерацией число потенциальных PIF уменьшается из-за более жестких накладываемых ограничений; по достижении count(PIF) < min_pixels работа алгоритма прерывается, считаем, что алгоритм не сошелся;
* max_iteration максимально допустимое число итераций, при превышении этого параметра, считаем, что алгоритм не сошелся. 

In [ ]:
def find_pif(grass, rast1, rast2, result, 
             diff_eps=0.2, diff_eps_rate=0.9,
             l_eps=0.01, l_eps_rate=0.9,
             regression_eps=0.01,
             min_pixels=1000,
             max_iteration=25):
    
    ###### Params for flow control and quality assesment #########
    # diff_eps = 0.2  # Init threshold for the difference between input rasters
    # diff_eps_rate = 0.9  # Rate for decrease diff_eps param during every iteration.
    
    # l_eps = 0.01  # Init threshold for l (l < minor < l)
    # l_eps_rate = 0.9  # Rate for decrease l_eps param during every iteration.
    
    # Tolerance for regression line coef: abs(b - 1) < regression_eps
    # where PIF_rast1 = a + b*PIF_rast2; b(very_good_quality) == 1
    # regression_eps = 0.001 
    
    # If count (unmasked pixels) < then min_pixels, then exit. (good_quality := False)
    # min_pixels = 1000   
    
    # max_iteration = 25  # Maximum count of iterations. 
    
    ###### End of params for flow control and quality assesment #########
    
    
    good_quality = False  # Are the PIFs good?
    
    current_iteration = 0
    while (not good_quality) and (current_iteration < max_iteration):
        current_iteration += 1
        try:
            grass.run_command('r.mask', flags='r')
        except:
            pass

        # Step 1
        try:
            tmp_diff_name = 'tmp' + uuid.uuid4().hex
            expr = '%s = abs(%s - %s)' % (tmp_diff_name, rast1, rast2)
            grass.run_command('r.mapcalc', expression=expr)
            grass.run_command('r.mapcalc', 
                              expression="MASK=if(%s < %s, 1, 0)" % (tmp_diff_name, diff_eps))
        finally:
            grass.run_command('g.remove', type='rast', name=tmp_diff_name, flags='f')
    
        try:
            # Step 2
            tmp_pca_name = 'tmp' + uuid.uuid4().hex
            grass.run_command('i.pca',
                              input=','.join([rast1, rast2]),
                              output=tmp_pca_name,
                              rescale='0,0'
                             )
            # Step 3
            grass.run_command('r.mask', flags='r')
            # PCA minor is stored in 'xxxxx.2' raster (we have 2 rasters only)
            expr = '%s = if(abs(%s) > %s, 0, 1)' % ('MASK', tmp_pca_name+'.2', l_eps)
            grass.run_command('r.mapcalc', expression=expr)
        finally:
            grass.run_command('g.remove', type='rast', pattern=tmp_pca_name+'*', flags='f')
    
        # Step 4
        # regression line quality:
        coefs = grass.read_command('r.regression.line',
                          mapx=rast1, mapy=rast2, flags='g'
                        )
        for line in coefs.split():
            if 'b=' == line[:2]:
                b = float(line[2:])
            elif 'N=' == line[:2]:
                N = int(line[2:])

        if N < min_pixels:
              # Next iterations don't increase count of umnasked pixels
            return {'good_quality': False, 
                    'iterations': current_iteration,
                    'N': N
                   }
        
        if abs(b - 1) < regression_eps:
            # PIF are found
            good_quality = True
            grass.run_command('g.copy', rast=','.join(['MASK', result]), overwrite=True)
            
        diff_eps *= diff_eps_rate
        l_eps *= l_eps_rate
    
    grass.run_command('r.mask', flags='r')
    
    result_info = dict(
        good_quality=good_quality,
        N=N,
        b=b,
        iterations=current_iteration
    )
                              
    return result_info



In [ ]:
import os
import uuid

import utilites
reload(utilites)

from utilites import (
    get_grassdata_path,
    get_location_name,
    get_ll_location_name,
    get_location_path,
)


from grasslib import GRASS

grs = GRASS(gisbase='/usr/lib/grass70', 
            dbase=get_grassdata_path(), 
            location=get_location_name()
)

grs.grass.run_command('g.mapset', mapset='landsat', flags='c')
print grs.grass.read_command('g.mapset', flags='p')

Пример запуска:

In [ ]:
rast1 = 'clean.topo.toar_LC81120272014106LGN00_B2'
rast2 = 'clean.topo.toar_LC81120272014090LGN00_B2'
grs.grass.run_command('g.region', rast=rast1, res='120')

find_pif(grs.grass, rast1, rast2, 'tmp_PIF')

Напишем функцию, которая будет брать растр, PIF, желаемые величины среднего и дисперсии и возвращать коэффициенты нормализации intercept и factor ($N_{norm} = factor \times N + intercept$):

In [ ]:
def get_norm_coef(grass, rast, pif, mean_ref=0.0, var_ref=1.0):
    try:
        grass.run_command('r.mask', raster=pif)
        stat = grass.read_command('r.univar', map=rast, flags='g')
    finally:
        grass.run_command('r.mask', flags='r')
        
    stat = stat.split()
    stat = dict([s.split('=') for s in stat])

    var = float(stat['variance'])
    mean = float(stat['mean'])
    
    factor = var_ref/var
    res = dict(
        intercept=(mean_ref - mean) * factor,
        factor=factor
    )
    
    return res

In [ ]:
print get_norm_coef(grs.grass, rast1, 'tmp_PIF')
print get_norm_coef(grs.grass, rast2, 'tmp_PIF')

### Подбор параметров нормализации

В алгоритме поиска PIF используются несколько параметров. Наибольшее влияние на результат должны оказывать, в первую очередь, параметры, определяющие критерии остановки работы алгоритма: regression_eps, min_pixels, max_iteration.

В этом разделе изучается достоверность нормирующих коэффициентов и влияние параметров алгоритма на коэффициенты нормализации. В качестве примера возмем одну сцену (path/row) и один канал.

Начнем с того, что вычислим нормирующие коэффициенты для одного канала одной сцены всеми доступными альтернативными способами. Затем постараемся понять, насколько сильно отличаются полученные коэффициенты и почему.

In [ ]:
prefix = 'clean.topo.toar_LC8112027'
rasters = grs.grass.list_strings(type='rast', pattern=prefix+'*_B1')

Результат выполнения команды очень многословен, см. его [в приложении 1](#appendix1). Для нас там важно то, что:
 
 * часть коэффициентов не была найдена из-за недостаточного количества итераций, соответственно, при большем числе допустимых итераций или меньшей требуемой точности эти коэффициенты можно было бы определить;
 * часть коээфициентов не была найдена из-за недостаточного количества PIF. 

Посмотрим на полученные нормировочные коэффициенты. "Сырые" данные лежат в [в приложении 2](#appendix2), для удобства анализа сведем их в таблицу.

В строках таблицы будут перечислены нормируемые растры, в заголовках столбцов - растры, которые использовались для поиска PIF совместно с растром, указанном в строке. В ячейке лежат полученные коэффициенты нормирования, вычисленные по PIF для указанной пары растров. Если для данной пары растров алгоритм поиска PIF не сошелся, в ячейке стоит прочерк.

Ожидается, что коэффициенты нормирования растра должны быть приблизительно одинаковыми не зависимо от второго растра использованного при поиске PIF. Т.е. в каждой строке должны быть приблизительно одинаковые значения.

|LC81120272014074|LC81120272014090|LC81120272014106|LC81120272014346|LC81120272014362|LC81120272015045|LC81120272015077|LC81120272015093|LC81120272015109|LC81120272015157|LC81120272015189|LC81120272015333|LC81120272015349|LC81120272015365|LC81120272016032|LC81120272016048|LC81120272016064|LC81120272016080|LC81120272016096
-|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|-
LC81120272014074|-|f=163.867578279; i=-32.4920630756|-|f=49.5817879123; i=-21.1954656682|f=62.4287105015; i=-26.6610331836|f=29.5464527696; i=-10.3111348373|f=38.5220610646; i=-17.4218223919|-|f=61.6918746854; i=-16.0135007081|f=1019.43713808; i=-165.86428063|f=3476.95458396; i=-518.415259479|f=66.8879278799; i=-20.65836751|-|f=48.2871331745; i=-12.37328029|f=33.9422210989; i=-8.71672105288|f=28.3601455449; i=-13.5554404049|f=29.0904929306; i=-10.4583804396|f=29.5374878342; i=-9.51844262607|-
LC81120272014090|f=163.535015844; i=-28.4644780449|-|f=175.436369355; i=-26.3803216807|-|-|-|-|-|f=102.123118397; i=-16.3157581448|-|-|-|-|-|-|f=431.620544375; i=-63.6344353405|f=408.230734907; i=-61.390740139|f=388.65660741; i=-59.0144629906|f=1547.2438965; i=-218.017240383
LC81120272014106|-|f=175.051381076; i=-28.2066893245|-|-|-|-|-|f=327.879604864; i=-229.361745738|f=32.0368069653; i=-7.23102936654|f=405.096549417; i=-63.2214819873|-|-|-|f=74.3497226378; i=-15.4993281438|f=117.295218765; i=-21.0979279468|f=105.388695748; i=-19.4094843427|f=102.887670929; i=-19.0686474855|f=133.275748545; i=-22.7258536295|f=471.511335069; i=-72.8130591757
LC81120272014346|f=50.5167351188; i=-21.5654308759|-|-|-|f=96.9295372481; i=-34.0616877476|f=85.7093266812; i=-26.9129523501|-|-|-|-|f=1220.43811792; i=-315.497343597|f=154.856462433; i=-47.6395397343|f=380.12887884; i=-131.058706572|f=60.8135589062; i=-23.3647092709|-|-|-|-|-
LC81120272014362|f=63.6055532139; i=-27.1865722891|-|-|f=98.8486719714; i=-35.0591594848|-|-|f=54.1460964044; i=-24.4192320228|-|-|-|-|f=178.678835227; i=-53.7582533744|f=349.746303598; i=-117.519509416|-|-|f=38.5412436359; i=-17.0933691678|f=62.7115586009; i=-26.8164562479|-|-
LC81120272015045|f=29.0099428187; i=-10.4989260664|-|-|f=84.2974969497; i=-26.4129725531|-|-|f=35.261096912; i=-13.5238946647|-|-|-|-|-|-|-|f=27.440792025; i=-9.5354600208|-|f=29.6385811516; i=-10.3750592693|f=33.9275688387; i=-11.543617241|-
LC81120272015077|f=39.2021015877; i=-17.77183017|-|-|-|f=53.1685901434; i=-23.9541118789|f=35.9348430805; i=-13.3546617371|-|-|-|-|-|-|-|f=44.7607741718; i=-13.6618546145|-|-|-|f=34.6016722487; i=-12.9383645722|f=269.655489301; i=-54.2320666525
LC81120272015093|-|-|f=331.050377813; i=-242.015534022|-|-|-|-|-|-|f=404.688845142; i=-287.591320049|-|-|-|-|-|-|-|f=67.6451335053; i=-43.7921195882|-
LC81120272015109|f=60.7100414218; i=-16.0147417423|f=104.290579096; i=-19.4149585129|f=32.5101443891; i=-7.81821861542|-|-|-|-|-|-|-|-|-|-|f=96.1958708105; i=-19.9128590727|f=191.633578653; i=-33.5201451948|f=179.750598788; i=-31.4486023613|f=167.855598277; i=-29.836467606|f=77.5837283973; i=-16.1768560038|f=987.857165423; i=-158.387909748
LC81120272015157|f=1035.18071679; i=-128.381388158|-|f=406.234733601; i=-47.1886957479|-|-|-|-|f=408.789776028; i=-286.844293884|-|-|f=4392.87744308; i=-502.160973641|-|-|-|f=1079.85834975; i=-131.609031038|-|f=721.688975812; i=-89.4078510249|f=944.412374824; i=-113.923275486|-
LC81120272015189|f=3494.53398776; i=-337.316075919|-|-|f=1304.15916561; i=-139.167769469|-|-|-|-|-|f=7562.99402867; i=-783.787387693|-|f=872.412610714; i=-96.2565262299|-|-|-|-|-|-|-
LC81120272015333|f=68.093646362; i=-21.0905817673|-|-|f=157.209887998; i=-45.8713824372|f=175.696701022; i=-51.645878369|-|-|-|-|-|f=956.209922748; i=-212.175285813|-|f=230.228623388; i=-69.3075963055|f=97.260610411; i=-27.0220801877|-|-|-|-|-
LC81120272015349|-|-|-|f=386.806262604; i=-133.662604501|f=356.285441142; i=-120.950377603|-|-|-|-|-|-|f=234.357406535; i=-71.0227313942|-|-|-|-|-|-|-
LC81120272015365|f=49.2444835736; i=-12.6763732198|-|f=75.4696629155; i=-15.8978954237|f=59.8232440067; i=-20.5015532426|-|-|f=45.5453764308; i=-13.8398242237|-|f=97.8466944313; i=-20.419303055|-|-|f=95.5574013889; i=-26.4381054496|-|-|-|f=85.9007043954; i=-20.1459686185|-|-|f=348.180278256; i=-68.9051860694
LC81120272016032|f=34.5868493928; i=-8.91903946756|-|f=119.349310118; i=-21.7531183922|-|-|f=27.8387823701; i=-8.51752900083|-|-|f=195.464416693; i=-34.417421172|f=1159.64382758; i=-200.445095362|-|-|-|-|-|-|-|f=31.4927380777; i=-9.48201116925|f=634.308306637; i=-105.493900125
LC81120272016048|f=27.895379328; i=-13.3458420977|f=447.375136908; i=-81.8843550891|f=107.176958474; i=-20.0894513257|-|f=37.7973114735; i=-16.7339400959|-|-|-|f=183.151282389; i=-32.4955657763|-|-|-|-|f=84.5728481056; i=-19.8082419086|-|-|f=35.2866877745; i=-9.01601665407|f=24.6670400578; i=-7.72734683239|f=409.256224599; i=-76.7229319875
LC81120272016064|f=28.4352469275; i=-10.13859846|f=410.274742929; i=-73.1195055859|f=104.763509211; i=-19.7163574053|-|f=61.4854394967; i=-26.2429167096|f=29.7724958749; i=-9.0104919125|-|-|f=171.060632259; i=-30.6866406511|f=761.027147305; i=-134.969901347|-|-|-|-|-|f=35.9422725646; i=-9.10726864729|-|f=32.0598231268; i=-9.55919915002|f=473.62547028; i=-83.1218744393
LC81120272016080|f=30.1290529448; i=-9.68002136394|f=389.275348336; i=-66.2115916116|f=135.635299112; i=-23.4803281565|-|-|f=34.3999481832; i=-9.80797866767|f=33.9597407326; i=-12.6397989311|f=68.6980530842; i=-40.653926673|f=79.4747626853; i=-16.5112364381|f=1002.68290872; i=-165.823358056|-|-|-|-|f=32.1072514748; i=-9.44998079616|f=25.0080504897; i=-7.4685525508|f=32.5408376616; i=-9.42802083279|-|-
LC81120272016096|-|f=1570.37615845; i=-242.823918401|f=480.33972724; i=-75.4359530339|-|-|-|f=264.992222435; i=-52.6940072354|-|f=997.476618888; i=-153.549005504|-|-|-|-|f=349.613512144; i=-63.3932067007|f=646.640513632; i=-109.527500292|f=414.054075026; i=-74.6658192888|f=482.446860743; i=-83.9421063272|-|-

Как мы видим, величины полученных коэффициентов далеки от того, чтобы быть близкими. Например, нормирующие множители одного и того же растра могут отличаться даже не в десятки, а в сотни раз! Как следствие, **в имеющемся виде данная процедура нормирования неприменима** и нужно понять, что с ней не так.

<a id='appendix1'></a>
### Приложение 1: лог поиска PIF

<a id='appendix2'></a>
### Приложение 2